The output from the Skyline document (if using a novel document and not the preformed human/conserved document) does not include Peptide Notes with the corresponding histone mark (e.g. H3 K27me1). I want to use the hard-coded table of histone marks and their respective mass shifts provided in the complementary `generate_hptms.ipynb` script to reverse-engineer the Skyline Peptide Modified Sequence back into biological histone mark notation (e.g. `PEK[+42.04695]TIDER` would become `H3 K3me1`).


input: a CSV with Peptide Modified Sequences from `Skyline>File>Export>Custom Report...` formats

output: a new column with the Peptide Note using "histone mod" language (H3K27me1, H3K4ac, etc)

In [79]:
import sys
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
import numpy as np
from Bio import SeqIO
from pyteomics import fasta, parser, mass, achrom, electrochem, auxiliary
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import itertools

sys.stdout.write("Imported required packages successfully.\n")

# set the master table for modification mass shifts
MZSHIFT_DICT = {'[n-term PR]':'[+56]',
                '[AC]': '[+42]',
                '[PR]': '[+56]',
                '[ME1]': '[+14]', 
                '[ME1+PR]': '[+70]',  # methyl+propionyl
                '[ME2]': '[+28]',
                '[ME3]': '[+42]',
                '[PH]': '[+80]',
                '[n-term ME3+PR]': '[+98.1]',
                '[n-term ME2+PR]': '[+84.1]',
                '[n-term ME1+PR]': '[+126.1]',
                '[n-term PR2]': '[+112.1]',
                '[n-term AC]': '[+98]',
                '[n-term AC+PR]': '[+84.1]',
                '[GGprop+PR]': '[+170.1]'}


##
## read input files: FASTA and Skyline Export Report with Peptide Modified Sequences
##

# read in FASTA
fasta_file = os.path.join(os.getcwd(), 
                          "../../collab_greer/data/uniprot-dicty_histones_7entries_acc20201231.fasta")

protein_df = pd.DataFrame()  # Initialize a dataframe to store results
for protein in tqdm(SeqIO.parse(fasta_file, "fasta")):
    
    protein_sequence = str(protein.seq).upper()
    
    # cleave initial "start" methionine if present
    if protein_sequence[0] == "M":
        protein_sequence = protein_sequence[1:]

    # add this protein to the dataframe
    new_df = pd.DataFrame({'protein': protein.id,
                           'protein_sequence': protein_sequence}, index=[0])
    
    protein_df = protein_df.append(new_df)
protein_df = protein_df.drop_duplicates()


# read in Skyline Export Report with Peptide Modified Sequences
skyline_df = pd.read_csv(os.path.join(os.getcwd(), 
                                      "../../collab_greer/data/greer_onlyhistlibrary_groupcomparison_mound-v-vegetative.csv"))

##
## "decode" modified peptide sequences to biological histone marks
##


# remove propionylations ([+56], [+112.1]) which aren't biologically relevant here
skyline_df['new_pep_seq'] = skyline_df['Peptide Modified Sequence']
skyline_df['new_pep_seq'] = skyline_df['new_pep_seq'].str.replace(r'\[\+56\]', '')
skyline_df['new_pep_seq'] = skyline_df['new_pep_seq'].str.replace(r'\[\+112.1\]', '')

# decode each modified peptide sequence to its modified residue number and mod type
decode_df = pd.DataFrame()  # Initialize a dataframe to store results
for index, row in skyline_df.iterrows():
    peptide = row['Peptide']
    mod_seq = row['new_pep_seq']
    fc = row['Fold Change Result']
    pval = row['Adjusted P-Value']
    
    # find all protein matches for the peptide in case there are duplicate/non-unique peptides
    protein_match = list(protein_df[protein_df['protein_sequence'].str.contains(peptide)]['protein'])
    
    # map each peptide to its residue position in the protein and change mass shift to modification
    for protein in protein_match:
        sequence = protein_df[protein_df['protein'] == protein]['protein_sequence'][0]
        aa_index = sequence.find(peptide)  # get the amino acid index position
        
        # split the peptide sequence into residues including any [+nn] mod shift
        residue_list = re.sub( r"([A-Z])", r" \1", mod_seq).split()

        # build the [Residue][Index Position][modification] histone mark
        histone_mod = ''
        for i in range(len(residue_list)): 
            if '[' in residue_list[i]:
                aa_pos = aa_index + i + 1  # have to +1 for indexing
                
                # match the mass shift to the mod and format to make it pretty
                mod = list(MZSHIFT_DICT.keys())[list(MZSHIFT_DICT.values()).index(residue_list[i][1:])]
                mod = re.sub('n-term ', '', mod)
                mod = re.sub(r'\+PR', '', mod)
                mod = mod.lower()  
                
                # build [Residue][Index Position][modification] string
                new_mod = residue_list[i][0] + str(aa_pos) + mod
                histone_mod = histone_mod + new_mod
        
        # add this protein/peptide to the dataframe
        new_df = pd.DataFrame({'Protein Name': protein,
                               'Peptide Sequence': peptide,
                               'Peptide Modified Sequence': row['Peptide Modified Sequence'],
                               'histone mark': histone_mod,
                               'Fold Change Result': fc,
                               'Adjusted P-Value': pval}, index=[0])
        decode_df = decode_df.append(new_df)
        
decode_df = decode_df.drop_duplicates()

decode_df = decode_df.groupby(['Peptide Modified Sequence', 'histone mark', 
                               'Fold Change Result', 'Adjusted P-Value'])['Protein Name'].apply(lambda x: ','.join(x)).reset_index()

decode_df.to_csv("D:/Penn/proj/collab_greer/data/greer_onlyhistlibrary_groupcomparison_mound-v-vegetative_decoded.csv", 
                        index=False)

decode_df.head(50)

7it [00:00, 1175.91it/s]

Imported required packages successfully.


,Peptide Modified Sequence,histone mark,Fold Change Result,Adjusted P-Value,Protein Name
0,A[+56]GITFPVSR,,1.72 (95% CI:0.63 to 4.7),0.4565,sp|Q54WG6|H2AX_DICDI
1,A[+56]LKR,,0.32 (95% CI:0.05 to 2),0.3984,"sp|P54671|H1_DICDI,sp|Q76NW2|H4_DICDI"
2,DIQVAVR,,177.48 (95% CI:0.01 to 2557464.47),0.4565,sp|Q54LP8|H2BV3_DICDI
3,D[+56]IQVAVR,,1.17 (95% CI:0.25 to 5.53),0.8790,sp|Q54LP8|H2BV3_DICDI
4,D[+56]SVAYTEHAGR,,0.2 (95% CI:0.06 to 0.65),0.0853,sp|Q76NW2|H4_DICDI
5,D[+56]SVAYTEHAGRR,,0.01 (95% CI:0 to 0.12),0.0461,sp|Q76NW2|H4_DICDI
6,EIAQEFKTDLR[+14],R83[me1],1.03 (95% CI:0 to 526.05),0.9910,sp|Q55BN9|H33B_DICDI
7,EIAQEFKTDLR[+14],R86[me1],1.03 (95% CI:0 to 526.05),0.9910,sp|O15819|H33A_DICDI
8,EIAQEFK[+28]TDLR,K79[me2],1.08 (95% CI:0 to 680.77),0.9898,sp|Q55BN9|H33B_DICDI
9,EIAQEFK[+28]TDLR,K82[me2],1.08 (95% CI:0 to 680.77),0.9898,sp|O15819|H33A_DICDI
